In [1]:
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import math
from sklearn.metrics import confusion_matrix 
from scipy import stats
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

import statsmodels.api as sm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# read data
claim_df =  pd.read_csv("./data/ClaimLevel.csv")
policy_df = pd.read_csv("./data/PolicyLevel.csv")

"""
                        # Data Preparation & Feature generation#
Generate a single record per policy that retains max information possible
"""
claim_df['CustomerPaidAmount'] = claim_df['ClaimedAmount'] - claim_df['PaidAmount']
claim_df_grouped = claim_df.groupby(['PolicyId'])
claim_per_policy_df = claim_df_grouped[['ClaimedAmount', 'PaidAmount', 'CustomerPaidAmount']].sum()
claim_per_policy_df['ClaimsCount'] = claim_df_grouped.size()
# claim_per_policy_df['AvgClaimAmount'] = claim_df_grouped[['ClaimedAmount']].mean()
# claim_per_policy_df['AvgPaidAmount'] = claim_df_grouped[['PaidAmount']].mean()
claim_per_policy_df['AvgCustomerPaidAmount'] = claim_df_grouped[['CustomerPaidAmount']].mean()
# claim_per_policy_df['stdClaimAmount'] = claim_df_grouped[['ClaimedAmount']].std()
# claim_per_policy_df['stdPaidAmount'] = claim_df_grouped[['PaidAmount']].std()
# claim_per_policy_df['stdCustomerPaidAmount'] = claim_df_grouped[['CustomerPaidAmount']].std()
# claim_per_policy_df['VarianceClaimAmount'] = claim_df_grouped[['ClaimedAmount']].var()
# claim_per_policy_df['VariancePaidAmount'] = claim_df_grouped[['PaidAmount']].var()
# claim_per_policy_df['VarianceCustomerPaidAmount'] = claim_df_grouped[['CustomerPaidAmount']].var()
claim_per_policy_df['maxClaimAmount'] = claim_df_grouped[['ClaimedAmount']].max()
# claim_per_policy_df['minClaimAmount'] = claim_df_grouped[['ClaimedAmount']].min()
# claim_per_policy_df['medianClaimAmount'] = claim_df_grouped[['ClaimedAmount']].median()
claim_per_policy_df['maxCustomerPaidAmount'] = claim_df_grouped[['ClaimedAmount']].max()
# claim_per_policy_df['minCustomerPaidAmount'] = claim_df_grouped[['ClaimedAmount']].min()
# claim_per_policy_df['medianCustomerPaidAmount'] = claim_df_grouped[['ClaimedAmount']].median()

policy_df = policy_df.join(claim_per_policy_df, on=['PolicyId'])

# Last seen Date: CancelDate if not null else '2016-12-31' (Last date in the data set)
policy_df['LastSeenDate'] = policy_df['CancelDate']
policy_df['LastSeenDate'].fillna(pd.to_datetime('2016-12-31'), inplace=True)

# Date conversions
policy_df['EnrollDate'] = pd.to_datetime(policy_df['EnrollDate'])
policy_df['CancelDate'] = pd.to_datetime(policy_df['CancelDate'])
policy_df['LastSeenDate'] = pd.to_datetime(policy_df['LastSeenDate'])

# Length of Stay "LOS"
policy_df['LOS'] = ((policy_df['LastSeenDate'] - policy_df['EnrollDate'])/np.timedelta64(1, 'M'))
policy_df = policy_df.round(decimals=2)

# Insurance Status: Active means 1 else 0
policy_df['InsuranceStatus'] = policy_df['CancelDate'].apply(lambda x: 0 if pd.notnull(x) else 1)

# drop null values
policy_df.dropna(axis='index', subset=['MonthlyPremium'], inplace=True)

# If no claims were made then all amounts are equal to 0 instead of NULL
for column in policy_df.columns:
    policy_df[column].fillna(0, inplace=True)

# display(claim_df.head())
display(policy_df.head(6))

,PolicyId,EnrollDate,CancelDate,MonthlyPremium,ClaimedAmount,PaidAmount,CustomerPaidAmount,ClaimsCount,AvgCustomerPaidAmount,maxClaimAmount,maxCustomerPaidAmount,LastSeenDate,LOS,InsuranceStatus
0,92597,2010-12-07,1970-01-01,34.54,0.00,0.00,0.00,0.0,0.00,0.00,0.00,2016-12-31,72.81,1
1,92808,2010-11-09,1970-01-01,18.54,0.00,0.00,0.00,0.0,0.00,0.00,0.00,2016-12-31,73.73,1
2,93090,2010-11-11,1970-01-01,23.53,762.49,329.46,433.03,3.0,144.34,510.48,510.48,2016-12-31,73.66,1
3,93122,2010-11-11,1970-01-01,44.02,0.00,0.00,0.00,0.0,0.00,0.00,0.00,2016-12-31,73.66,1
4,93133,2010-11-11,1970-01-01,32.46,0.00,0.00,0.00,0.0,0.00,0.00,0.00,2016-12-31,73.66,1
5,93258,2010-11-11,2016-05-24,21.66,33.99,30.60,3.39,3.0,1.13,11.33,11.33,2016-05-24,66.40,0


In [3]:
"""
                   # Data Description #
Churn --> 'C' 
Non-Churn --> 'NC'
"""
total_policies = policy_df.shape[0]
curr_NC = policy_df[policy_df['InsuranceStatus']==1].shape[0]/float(total_policies)
curr_C = policy_df[policy_df['InsuranceStatus']==0].shape[0]/float(total_policies)
print("Total polices", total_policies)
print("NC:C = ",curr_NC, ":", curr_C)

Total polices 99995
NC:C =  0.8678433921696085 : 0.13215660783039151


In [4]:
# Sampling Data
churn_indices = policy_df[policy_df['InsuranceStatus']==0].index.values
nonchurn_indices = policy_df[policy_df['InsuranceStatus']==1].index.values
# print(len(churn_indices), len(nonchurn_indices))

def sample_train_data(num_samples=None, churn_percent=None):
    """
    :param num_samples: total number of samples to return
    :param nc: ratio of non-churn in the return data set
    :param c: ration of churn in the return data set
    """
    # check to see if nc + c = 1
    churn_sample_space = churn_indices
    non_churn_sample_space = nonchurn_indices
        
    if num_samples is None and churn_percent is None:
        # use max samples possible (only 80% for train)
        churn_percent = curr_C 
        num_samples = int(len(churn_indices)*0.8/churn_percent)
    elif num_samples is None:
        num_samples = int(len(churn_indices)*0.8/churn_percent)
    
    if churn_percent==None:
        # default rate
        churn_percent = curr_C 
    elif churn_percent > 1:
        raise ValueError('Churn percent cannot be > 1')
    
    non_churn_percent = 1 - churn_percent
    nc_samples_count = int(num_samples * non_churn_percent)
    c_samples_count = int(num_samples * churn_percent)
    
    
    # prune train indices if looking for test set
    churn_sample_indices = np.random.choice(churn_sample_space, size=c_samples_count, replace=False)
    nonchurn_sample_indices = np.random.choice(non_churn_sample_space, size=nc_samples_count, replace=False)
    sample_indices = np.concatenate((churn_sample_indices, nonchurn_sample_indices))
    # shuffle
    np.random.shuffle(sample_indices)
    sample = policy_df.loc[sample_indices, :]
    return sample

def sample_test_data(skip_indices=[]):
    
    churn_sample_space = np.setdiff1d(churn_indices, skip_indices)
    non_churn_sample_space = np.setdiff1d(nonchurn_indices, skip_indices)
    churn_sample_indices = churn_sample_space
    nonchurn_sample_indices = np.random.choice(non_churn_sample_space, size=len(churn_sample_indices), replace=False)
    sample_indices = np.concatenate((churn_sample_indices, nonchurn_sample_indices))
    # shuffle
    np.random.shuffle(sample_indices)
    sample = policy_df.loc[sample_indices, :]
    return sample
    
# test case
test = True
if test == False:
#     sample = sample_data(100, 0.8, 0.2)
    sample = sample_data()
    sample.head()
    sample_total_policies = sample.shape[0]
    NC = sample[sample['InsuranceStatus']==1].shape[0]/float(sample_total_policies)
    C = sample[sample['InsuranceStatus']==0].shape[0]/float(sample_total_policies)
    print("Total polices", total_policies)
    print("NC:C = ",NC, ":", C)

In [5]:
# From the train data evaluation this the best set
# # data set 2: NC:C = 1:1 and contains max samples possible
train_set2_df = sample_train_data(churn_percent=0.5)
test_set2_df = sample_test_data(skip_indices=train_set2_df.index.values)

In [6]:
select_columns = list(
    set(policy_df.columns) - set(['PolicyId', 'EnrollDate', 'CancelDate', 'InsuranceStatus', 'LastSeenDate',
#                                  'ClaimedAmount', 'PaidAmount', 'CustomerPaidAmount' , 
#                                  'AvgClaimAmount', 'AvgPaidAmount', 'stdClaimAmount', 'stdPaidAmount'
#                                   'minClaimAmount', 'maxClaimAmount', 'medianClaimAmount'
                                 ]))

datasets  = [
        [train_set2_df, test_set2_df],
        ]

In [9]:
"""
                                        # RF Model #
"""
from sklearn.ensemble import RandomForestClassifier as RF

for dataset in datasets:
    X_train = dataset[0][select_columns]
    X_test = dataset[1][select_columns]
    y_train = dataset[0][['InsuranceStatus']]
    y_test = dataset[1][['InsuranceStatus']]
    X_train, X_test = scaler.fit_transform(X_train), scaler.fit_transform(X_test)
    model = RF(n_estimators=100, class_weight={0:3,1:1})
    model.fit(X_train, y_train)
    y_predicted = model.predict(X_test)
    print("Test accuracy",model.score(X_test, y_test))
    print("tn, fp, fn, tp ",confusion_matrix(y_test, y_predicted).ravel())


Test accuracy 0.578130911843
tn, fp, fn, tp  [1447 1196 1034 1609]


In [11]:
"""
                                        # Logistic Model #
"""
from sklearn.linear_model import LogisticRegression

for dataset in datasets:
    X_train = dataset[0][select_columns]
    X_test = dataset[1][select_columns]
    y_train = dataset[0][['InsuranceStatus']]
    y_test = dataset[1][['InsuranceStatus']]
    X_train, X_test = scaler.fit_transform(X_train), scaler.fit_transform(X_test)
    for weight in [0.5, 0.75, 1, 1.5, 2, 3, 4]:
        point_weights = dataset[0].InsuranceStatus.apply(lambda x: weight if x == 0 else 1)
        model = LogisticRegression(penalty='l1',)
        model.fit(X_train, y_train, point_weights)
        y_predicted = model.predict(X_test)
        print("Weight on disenrolled polcies", weight)
        print("Test Accuracy",model.score(X_test, y_test))
        print("tn, fp, fn, tp ",confusion_matrix(y_test, y_predicted).ravel())
#         X2 = sm.add_constant(X_train)
#         est = sm.Logit(y_train, X2)
#         est2 = est.fit()
#         print(est2.summary())

Weight on disenrolled polcies 0.5
Test Accuracy 0.501891789633
tn, fp, fn, tp  [  16 2627    6 2637]
Weight on disenrolled polcies 0.75
Test Accuracy 0.595724555429
tn, fp, fn, tp  [1088 1555  582 2061]
Weight on disenrolled polcies 1
Test Accuracy 0.59515701854
tn, fp, fn, tp  [1863  780 1360 1283]
Weight on disenrolled polcies 1.5
Test Accuracy 0.551645856981
tn, fp, fn, tp  [2346  297 2073  570]
Weight on disenrolled polcies 2
Test Accuracy 0.530457813091
tn, fp, fn, tp  [2505  138 2344  299]
Weight on disenrolled polcies 3
Test Accuracy 0.514566780174
tn, fp, fn, tp  [2593   50 2516  127]
Weight on disenrolled polcies 4
Test Accuracy 0.508891411275
tn, fp, fn, tp  [2615   28 2568   75]


In [80]:
"""
                                        # neural nets Model #
"""
from sklearn.neural_network import MLPClassifier

for dataset in datasets:
    X_train = dataset[0][select_columns]
    X_test = dataset[1][select_columns]
    y_train = dataset[0][['InsuranceStatus']]
    y_test = dataset[1][['InsuranceStatus']]
    X_train, X_test = scaler.fit_transform(X_train), scaler.fit_transform(X_test)
    model = MLPClassifier(solver='lbfgs', alpha=1e-4, hidden_layer_sizes=(16, 32, 24, 8), random_state=1, 
                         activation='relu')
    model.fit(X_train, y_train)
    y_predicted = model.predict(X_test)
#         print("\t Train",model.score(X_train, y_train))
    print("Test",model.score(X_test, y_test))
#     print(confusion_matrix(y_test, y_predicted))

Test 0.598751418842
